In [1]:
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import gym
from tensorflow.keras import models, layers
import random

2023-11-02 11:11:28.516519: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-02 11:11:28.604103: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-02 11:11:29.017816: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 11:11:29.017867: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 11:11:29.019905: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
env = gym.make("Breakout-v0", obs_type='grayscale')

/home/meliioko/atari/.venv/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment Breakout-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]
/home/meliioko/atari/.venv/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (210, 160)
  logger.warn(


In [6]:
def build_model(action_size):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(84, 84, 4)))
    model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model


In [ ]:
def preprocess_state(state):
    # Assuming the state is already grayscale
    # Resize to 84x84 and reshape to the expected input shape for the model
    processed_state = resize(state, (84, 84)).reshape(1, 84, 84, 1)
    # Stack 4 frames
    processed_state = np.repeat(processed_state, 4, axis=3)
    return processed_state

In [ ]:
def preprocess_frame(frame):
    # Convert frame to grayscale if it is not already (assuming the environment returns colored frames)
    # Resize, normalize, etc.
    # ... your preprocessing steps here ...
    processed_frame = resize(frame, (84, 84))  # This is a placeholder, replace with actual preprocessing
    return processed_frame

# Initialize frame stacker
frame_stacker = deque(maxlen=4)

# Function to stack frames
def stack_frames(stack, new_frame, is_new_episode):
    frame = preprocess_frame(new_frame)
    
    if is_new_episode:
        # Clear our stack
        stack.clear()
        # Because we're in a new episode, copy the same frame 4x
        for _ in range(4):
            stack.append(frame)
    else:
        # Append frame to deque, automatically removes the oldest frame
        stack.append(frame)
    
    # Stack the frames along the third dimension and return a new numpy array
    stacked_state = np.stack(stack, axis=2)
    return stacked_state

In [16]:
# Set parameters
N = 10000  # Replay memory capacity
M = 1000  # Number of episodes
T = 10000  # Max steps per episode
C = 1000  # Target network update frequency
epsilon = 1
epsilon_decay = 0.995
epsilon_min = 0.1
gamma = 0.9
action_size = env.action_space.n  # Number of actions
state_size = env.observation_space.shape[0]  # State size

# Initialize replay memory
D = deque(maxlen=N)


Q = build_model(action_size)
Q_hat = build_model(action_size)
Q_hat.set_weights(Q.get_weights())


In [9]:
Q.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 20, 20, 32)        8224      
                                                                 
 conv2d_4 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 3136)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               1606144   
                                                                 
 dense_3 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 1686180 (6.43 MB)
Trainable params: 1686

In [18]:
    state = env.reset()
    total_reward = 0
    for t in range(T):
        # Epsilon-greedy action selection
        if np.random.rand() <= epsilon:
            action = random.randrange(action_size)
        else:
            act_values = Q.predict(state)
            action = np.argmax(act_values[0])

        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward

        # Store transition in D
        D.append((state, action, reward, next_state, done))

        state = next_state

        # Check if the episode is done
        if done:
            print(f"Episode: {episode}/{M}, Score: {total_reward}")
            break

        # Train using a random minibatch from D
        if len(D) > 32:
            minibatch = random.sample(D, 32)
            for w, a, r, w_next, terminal in minibatch:
                target = r
                if not terminal:
                    target = (r + gamma * np.amax(Q_hat.predict(w_next)[0]))
                target_f = Q.predict(w)
                target_f[0][a] = target
                Q.fit(w, target_f, epochs=1, verbose=0)

    # Update epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    # Update target network
    if episode % C == 0:
        Q_hat.set_weights(Q.get_weights())

ValueError: in user code:

    File "/home/meliioko/atari/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2416, in predict_function  *
        return step_function(self, iterator)
    File "/home/meliioko/atari/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2401, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/meliioko/atari/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2389, in run_step  **
        outputs = model.predict_step(data)
    File "/home/meliioko/atari/.venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2357, in predict_step
        return self(x, training=False)
    File "/home/meliioko/atari/.venv/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/meliioko/atari/.venv/lib/python3.10/site-packages/keras/src/engine/